Exercise Horizontal Pod Autoscaler
==================================

![](https://d33wubrfki0l68.cloudfront.net/4fe1ef7265a93f5f564bd3fbb0269ebd10b73b4e/1775d/images/docs/horizontal-pod-autoscaler.svg)

Quelle [Horizontal Pod Autoscaler](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale/)
- - -

The horizontal pod autoscaler automatically scales the number of pods in a ReplicaSet, Deployment, or StatefulSet.

Scaling can be based on observed CPU utilization or using custom metrics. See [hier](https://kubernetes.io/docs/tasks/run-application/horizontal-pod-autoscale/#algorithm-details) for details.

The metrics come from a separately started [Metric Server](https://github.com/kubernetes-sigs/metrics-server).

In [ ]:
! kubectl get pods --namespace metrics

***
Test Image
---------------

To demonstrate the horizontal pod autoscaler, we'll use a custom Docker image based on the PHP Apache image. The Docker file has the following content:

    FROM php:5-apache
    COPY index.php /var/www/html/index.php
    RUN chmod a+rx index.php
    
It defines an index.php page that does some CPU intensive calculations:

    <?php
      $x = 0.0001;
      for ($i = 0; $i <= 1000000; $i++) {
        $x += sqrt($x);
      }
      echo "OK!";
    ?>
    
First, let's start the pod running the image and deploy it as a service:

In [ ]:
%%bash
cat <<%EOF% | kubectl apply -f -
apiVersion: v1
kind: Namespace
metadata:
  name: hpa
%EOF%

cat <<%EOF% | kubectl apply --namespace hpa -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: php-apache
spec:
  selector:
    matchLabels:
      run: php-apache
  replicas: 1
  template:
    metadata:
      labels:
        run: php-apache
    spec:
      containers:
      - name: php-apache
        image: k8s.gcr.io/hpa-example
        ports:
        - containerPort: 80
        resources:
          limits:
            cpu: 500m
          requests:
            cpu: 200m
---
apiVersion: v1
kind: Service
metadata:
  name: php-apache
  labels:
    run: php-apache
spec:
  ports:
  - port: 80
  selector:
    run: php-apache
%EOF%


### Creation of horizontal pod autoscaler

After the server is running, let's build the autoscaler.

The following command creates a horizontal pod autoscaler that manages between 1 and 10 instances of the above pod.

Broadly speaking, the HPA will increase and decrease the number of replicas (via deployment) to maintain an average CPU utilization across all pods of 50%.

In [ ]:
! kubectl autoscale deployment php-apache --namespace hpa --cpu-percent=50 --min=1 --max=10

We look at the current status of our resources:

In [ ]:
! kubectl get pods --namespace hpa --show-kind=true
! kubectl get hpa  --namespace hpa --show-kind=true 

### Increase load

Now we will see how the autoscaler reacts to increased load. We're going to start a container and send an infinite loop of queries to the PHP Apache service.

In [ ]:
%%bash
cat <<%EOF% | kubectl apply --namespace hpa -f -
apiVersion: v1
kind: Pod
metadata:
  labels:
    app.kubernetes.io/name: load-generator
  name: load-generator
spec:
  containers:
  - image: busybox
    name: busybox
    command: [ "/bin/sh", "-c", "--" ]
    args: [ "while true; do wget -q -O- http://php-apache; done" ]
%EOF%


With the Horizontal Pod Autoscaler, we should now see Load > 50%. The displayed `Last / 50` gives the number of pods that have to run at the end.

In [ ]:
! kubectl get pods --namespace hpa --show-kind=true
! kubectl get hpa  --namespace hpa --show-kind=true 

If the pod `load-generator` is deleted, the load is reduced and the number of `php-apache` pods is reduced again.

In [ ]:
! kubectl delete --namespace hpa pod/load-generator

In [ ]:
! kubectl get pods --namespace hpa --show-kind=true
! kubectl get hpa  --namespace hpa --show-kind=true 

***
Clean up

In [ ]:
! kubectl delete namespace hpa